In [1]:
# setup
from pyswmm import Simulation
import os, dask, time, shutil
from pyswmm.lib import DLL_SELECTION

In [4]:
# save absolute path of input file location
# this string is how we will tell pyswmm where to locate the files it needs to run its simulations
input_path = os.path.abspath("../probabilistic_python/input/swmm/input_")
print(input_path)

# save the path for the original dynamic link library
dll_path = DLL_SELECTION()
print(dll_path)

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_


In [6]:
'''
Makes arrangements for executing the following task:
    Run specified probabilistic simulations in SWMM from the specified .inp file. Save output in equivalently named .out file.
Inputs:
    i <integer> -indicates input folder id number (for now, will only ever be 1-5).-
Immediate Output:
    <dask.delayed.Delayed> -arrangements for executing the task when commanded at a future time (by the 'compute()' function)
Computable Output:
    <string> -message indicating that the simulation has been completed on the .inp file in the 'i' input folder.-
'''
# this decorator is the first step in using dask to parallelize swmm simulations
@dask.delayed
def delay_job(i):
    # create path to the .exe we are going to create
    lib_path = dll_path[:-4]+'-'+str(i)+dll_path[-4:]
    # create .exe file
    shutil.copyfile(dll_path, lib_path)
    # specify the directory with the file pyswmm needs by attaching the folder id to the rest of the folder's absolute path
    sim_folder = input_path + str(i)
    # specify the actual file pyswmm needs
    sim_file = os.path.join(sim_folder, r'JS_NPlesantCreek.inp')
    print("Simulation input file found:", sim_file)
    # specify the file that pyswmm will (over)write with output after running the probabilistic simulation
    binary_file = sim_folder + r'\JS_NPlesantCreek.out'
    # delete pre-existing .out, if present, in order to run swmm agreeably
    if os.path.exists(binary_file):
        print("Deleting current copy of <JS_NPlesantCreek.out> so new copy can be created.")
        os.remove(binary_file)
    # stagger starting times 2 sec apart
    time.sleep(2*i)
    # load the model {no interaction, write (binary) results to <JS_NPlesantCreek.out>, use the specified dll}
    sim = Simulation(inputfile=sim_file, reportfile=None, outputfile=binary_file, swmm_lib_path=lib_path)
    # simulate the loaded model
    sim.execute()
    # delete the .exe file
    #os.remove(lib_path)
    # a message to indicate success
    return("file " + str(i) + " simulated!")

In [7]:
# can it do multiple?
delayed_jobs = [delay_job(1),
                delay_job(2),
                delay_job(3),
                delay_job(4),
                delay_job(5)]
delayed_jobs

[Delayed('delay_job-5a84b3f3-f19d-414e-b0f3-498d5353cc38'),
 Delayed('delay_job-6bc567c4-3a9b-45ed-a6c5-556e598183b9'),
 Delayed('delay_job-2e658d5f-061b-4b00-a764-f1411886dfe1'),
 Delayed('delay_job-b2c17f62-bd12-4f4d-a40e-aebdf6972ca8'),
 Delayed('delay_job-7deb3c54-8386-42d2-907f-22cfcab42709')]

In [8]:
%%time
dask.delayed(print)(delayed_jobs).compute()

Simulation input file found:Simulation input file found: C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_4\JS_NPlesantCreek.inp
 C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_3\JS_NPlesantCreek.inpSimulation input file found: Simulation input file found: C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_5\JS_NPlesantCreek.inp
Deleting current copy of <JS_NPlesantCreek.out> so new copy can be created.
Simulation input file found: C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_2\JS_NPlesantCreek.inp
Deleting current copy of <JS_NPlesantCreek.out> so new copy can be created.

... SWMM Version 5.3.0.dev0
... SWMM Version 5.3.0.dev0
... SW

PySWMM can handle simultaneously running two parallel processes, as long as each one uses its own direct link library. 